#最新完成コード

In [ ]:
# ライブラリのインストール
!pip install flask flask-ngrok pyngrok flask-cors

from flask import Flask, request, send_file, jsonify, redirect
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask_cors import CORS
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time

# ngrokの認証トークンを設定
ngrok.set_auth_token("2xM5m67sPshbacPkiTIoQw5nnWb_7i5WdwHgUxf3TtKioJiG")

app = Flask(__name__)
# CORS(app, resources={r"/process_images": {"origins": ["http://127.0.0.1:5501"]}})
# 全てのオリジンからのリクエストを許可
CORS(app)

# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    print(f"Downloading image from: {url}, Status code: {response.status_code}")
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

@app.route('/process_images', methods=['POST'])
def process_images():
    try:
        print(f"Request files: {request.files}")
        # 受け取ったファイルを取得.2班からのblob画像データ
        if 'longshirt[]' not in request.files:
            print("No images provided in the request.画像が含まれていません。")
            return jsonify({"error": "No images provided."}), 400

        # longshirtのファイルをリストとして取得
        files = request.files.getlist('longshirt[]')  # 'longshirt[]'に変更
        print(f"Number of files received: {len(files)}")

        for file in files:
            print(f"File name: {file.filename}, Content type: {file.content_type}")

        if len(files) != 4:
            return jsonify({"error": "Invalid input, expected 4 images."}), 400



        image_points = [
            np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
            np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
            np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
            np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
        ]

        # 貼り付け先の画像のURL
        target_image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
        target_image = download_image(target_image_url)

        target_points_list = [
            np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
            np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
            np.array([[106, 1131],[0, 1203],[100, 1882],[487, 1885],[584, 1203],[342, 1082]]),
            np.array([[888, 1094],[699, 1203],[795, 1885],[1182, 1883],[1284, 1204],[1041, 1086]]),
        ]

        # 画像を処理する
        for i, file in enumerate(files):
            image = Image.open(file.stream)
            image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

            points = image_points[i]
            target_points = target_points_list[i]

            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 255)

            M, status = cv2.findHomography(points, target_points)

            transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))
            mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
            target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

        # 変更された画像を一時ファイルに保存
        modified_target_image_path = 'modified_image.jpg'
        cv2.imwrite(modified_target_image_path, target_image)

        # 生成された画像を返す
        return send_file(modified_target_image_path, mimetype='image/jpeg')


    except Exception as e:
          print(f"Error processing file {file.filename}: {e}")
          return jsonify({"error": f"Error processing file {file.filename}: {str(e)}"}), 500


# テスト用app
@app.route('/', methods=['GET'])
def home():
    return "Flask is running! APIテストOK"

# JSテスト用app
@app.route('/img', methods=['GET'])
def img():
    image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
    return redirect(image_url)

if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ

    # ngrokのURLを取得
    url_info = requests.get('http://localhost:4040/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")

    app.run(port=5000)


ConnectionError: HTTPConnectionPool(host='localhost', port=4040): Max retries exceeded with url: /api/tunnels (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x78eefc225d80>: Failed to establish a new connection: [Errno 111] Connection refused'))

下のdata = request.jsonパターンは、画像データがgithub等のどこかにある場合（テストデータ）。↑は2班のコード内でblob形態にした場合（完成品）

In [ ]:
# ライブラリのインストール
!pip install flask flask-ngrok pyngrok flask-cors

from flask import Flask, request, send_file, jsonify, redirect
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask_cors import CORS
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time

# ngrokの認証トークンを設定
ngrok.set_auth_token("2xM5m67sPshbacPkiTIoQw5nnWb_7i5WdwHgUxf3TtKioJiG")

app = Flask(__name__)
# CORS(app, resources={r"/process_images": {"origins": ["http://127.0.0.1:5501"]}})
# 全てのオリジンからのリクエストを許可
CORS(app)

# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    print(f"Downloading image from: {url}, Status code: {response.status_code}")
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

@app.route('/process_images', methods=['POST'])
def process_images():
    try:
        data = request.json
        print(f"Received data: {data}")

        if not data:
            return jsonify({"error": "No data provided."}), 400

        image_urls = data.get("longshirt")
        if not image_urls or len(image_urls) != 4:
            return jsonify({"error": "Invalid input, expected 4 image URLs."}), 400

        image_points = [
            np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
            np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
            np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
            np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
        ]

        # 貼り付け先の画像のURL
        target_image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
        target_image = download_image(target_image_url)

        target_points_list = [
            np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
            np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
            np.array([[106, 1131],[0, 1203],[100, 1882],[487, 1885],[584, 1203],[342, 1082]]),
            np.array([[888, 1094],[699, 1203],[795, 1885],[1182, 1883],[1284, 1204],[1041, 1086]]),
        ]

        # 画像を処理する
        for i, (url, target_points) in enumerate(zip(image_urls, target_points_list)):
            print(f"Downloading image from: {url}")
            image = download_image(url)
            print(f"Successfully downloaded image: {url}")

            points = image_points[i]
            print(f"Source points for {url}: {points}, Shape: {points.shape}")
            print(f"Target points: {target_points}, Shape: {target_points.shape}")

            if len(points) != len(target_points):
                print(f"Mismatch in points count for {url}: {len(points)} vs {len(target_points)}")
                return jsonify({"error": f"Points count mismatch for {url}"}), 400

            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 255)
            print(f"Mask shape: {mask.shape}, unique values in mask: {np.unique(mask)}")

            M, status = cv2.findHomography(points, target_points)
            print(f"Homography Matrix for {url}:\n{M}")

            transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))
            print(f"Transformed image shape: {transformed_image.shape}")

            mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
            target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

        # 変更された画像を一時ファイルに保存
        modified_target_image_path = 'modified_image.jpg'
        cv2.imwrite(modified_target_image_path, target_image)

        # 生成された画像を返す
        return send_file(modified_target_image_path, mimetype='image/jpeg')

    except Exception as e:
        print(f"Error occurred: {e}")
        return jsonify({"error": str(e)}), 500

# テスト用app
@app.route('/', methods=['GET'])
def home():
    return "Flask is running! APIテストOK"

# JSテスト用app
@app.route('/img', methods=['GET'])
def img():
    image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
    return redirect(image_url)

if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ

    # ngrokのURLを取得
    url_info = requests.get('http://localhost:4040/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")

    app.run(port=5000)


#API化する前のGitHub

In [ ]:
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image

# GitHubから画像をダウンロードする関数 (raw.githubusercontent.com を使う)
def download_image(url):
    # GitHubのURLをraw形式に変換
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

# 画像のGitHub URLをリストに分ける
image_urls = [
    "https://github.com/ta03-jp/practice/blob/main/longshirt_front.jpg",
    "https://github.com/ta03-jp/practice/blob/main/longshirt_back.png",
    "https://github.com/ta03-jp/practice/blob/main/longshirt_sleeve.jpg",
    "https://github.com/ta03-jp/practice/blob/main/longshirt_sleeve.jpg"
]

image_points = [
    np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
    np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
    np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
    np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
]

# 貼り付け先の画像のURL
target_image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
target_image = download_image(target_image_url)

# 画像が正しく読み込まれたか確認
if target_image is None:
    raise FileNotFoundError(f"画像が見つかりません: {target_image_url}")

# 貼り付け先の頂点の座標のリスト
target_points_list = [
    np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
    np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
    np.array([[106, 1131],[0, 1203],[116, 1978],[494, 1977],[584, 1203],[342, 1082]]),
    np.array([[888, 1094],[699, 1203],[739, 1974],[1172, 1978],[1284, 1204],[1041, 1086]]),
]

# 各元の画像に対して処理を行う
for url, points, target_points in zip(image_urls, image_points, target_points_list):
    # 元の画像をダウンロードする
    image = download_image(url)

    # 画像が正しく読み込まれたか確認
    if image is None:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

    # マスクを作成
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [points], 255)

    # 変換行列を計算
    M, _ = cv2.findHomography(points, target_points)

    # 元の画像を変換して貼り付け先の形状に合わせる
    transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))

    # 変換された画像を貼り付け先の画像に合成
    mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
    target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

# 変更された画像を保存する
modified_target_image_path = '/content/drive/MyDrive/goudouzemi/API/tample1-2.jpg'
cv2.imwrite(modified_target_image_path, target_image)

#API化

In [ ]:
!pip install Flask pyngrok
from flask import Flask, request, send_file, jsonify
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time

app = Flask(__name__)

# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

# APIエンドポイント
@app.route('/process_images', methods=['POST'])
def process_images():
    data = request.json
    image_urls = data.get("longshirt")

    if not image_urls or len(image_urls) != 4:
        return jsonify({"error": "Invalid input, expected 4 image URLs."}), 400

    # 貼り付け先の画像のURL
    target_image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
    target_image = download_image(target_image_url)

    target_points_list = [
        np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
        np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
        np.array([[106, 1131],[0, 1203],[100, 1882],[487, 1885],[584, 1203],[342, 1082]]),
        np.array([[888, 1094],[699, 1203],[795, 1885],[1182, 1883],[1284, 1204],[1041, 1086]])
    ]

    # 画像を処理する
    for url, target_points in zip(image_urls, target_points_list):
        image = download_image(url)

        # マスクを作成
        points = np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]])
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [points], 255)

        M, _ = cv2.findHomography(points, target_points)
        transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))

        mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
        target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

    # 変更された画像を一時ファイルに保存
    modified_target_image_path = 'modified_image.jpg'
    cv2.imwrite(modified_target_image_path, target_image)

    # 生成された画像を返す
    return send_file(modified_target_image_path, mimetype='image/jpeg')

if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ

    # ngrokのURLを取得
    url_info = requests.get('http://localhost:4040/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")

    app.run(port=5000)

#GitHub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image

# GitHubから画像をダウンロードする関数 (raw.githubusercontent.com を使う)
def download_image(url):
    # GitHubのURLをraw形式に変換
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

# 画像のGitHub URLをリストに分ける
image_urls = [
    "https://github.com/ta03-jp/practice/blob/main/longshirt_front.jpg",
    "https://github.com/ta03-jp/practice/blob/main/longshirt_back.png",
    "https://github.com/ta03-jp/practice3/blob/main/longshirt_sleeve.jpg",
    "https://github.com/ta03-jp/practice/blob/main/longshirt_sleeve.jpg"
]

image_points = [
    np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
    np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
    np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
    np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
]

# 貼り付け先の画像のURL
target_image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
target_image = download_image(target_image_url)

# 画像が正しく読み込まれたか確認
if target_image is None:
    raise FileNotFoundError(f"画像が見つかりません: {target_image_url}")

# 貼り付け先の頂点の座標のリスト
target_points_list = [
    np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
    np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
    np.array([[106, 1131],[0, 1203],[116, 1978],[494, 1977],[584, 1203],[342, 1082]]),
    np.array([[888, 1094],[699, 1203],[739, 1974],[1172, 1978],[1284, 1204],[1041, 1086]]),
]

# 各元の画像に対して処理を行う
for url, points, target_points in zip(image_urls, image_points, target_points_list):
    # 元の画像をダウンロードする
    image = download_image(url)

    # 画像が正しく読み込まれたか確認
    if image is None:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

    # マスクを作成
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [points], 255)

    # 変換行列を計算
    M, _ = cv2.findHomography(points, target_points)

    # 元の画像を変換して貼り付け先の形状に合わせる
    transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))

    # 変換された画像を貼り付け先の画像に合成
    mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
    target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

# 変更された画像を保存する
modified_target_image_path = '/content/drive/MyDrive/tample1.jpg'
cv2.imwrite(modified_target_image_path, target_image)

True

#座標のリストがちゃんとあるやつ

In [ ]:
!pip install Flask pyngrok
from flask import Flask, request, send_file, jsonify
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time

app = Flask(__name__)

# ngrokのトークンを設定
ngrok_token = "2xM5m67sPshbacPkiTIoQw5nnWb_7i5WdwHgUxf3TtKioJiG"  # ここにあなたのngrokトークンを入れてください
subprocess.run(['ngrok', 'authtoken', ngrok_token])

# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")
# APIエンドポイント
@app.route('/')
def index():
    return "Welcome to the image processing API! Use the /process_images endpoint."
def process_images():
    data = request.json
    image_urls = data.get("longshirt")

    if not image_urls or len(image_urls) != 4:
        return jsonify({"error": "Invalid input, expected 4 image URLs."}), 400

    image_points = [
        np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
        np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
        np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
        np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
    ]

    # 貼り付け先の画像のURL
    target_image_url = "https://ta03-jp.github.io/practice/longTshirts_frame_only.png"
    target_image = download_image(target_image_url)

    target_points_list = [
        np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
        np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
        np.array([[106, 1131],[0, 1203],[100, 1882],[487, 1885],[584, 1203],[342, 1082]]),
        np.array([[888, 1094],[699, 1203],[795, 1885],[1182, 1883],[1284, 1204],[1041, 1086]]),
    ]

    # 画像を処理する
    for url, points, target_points in zip(image_urls, image_points, target_points_list):
        image = download_image(url)

        # マスクを作成
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [points], 255)

        M, _ = cv2.findHomography(points, target_points)
        transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))

        mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
        target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

    # 変更された画像を一時ファイルに保存
    modified_target_image_path = 'modified_image.jpg'
    cv2.imwrite(modified_target_image_path, target_image)

    # 生成された画像を返す
    return send_file(modified_target_image_path, mimetype='image/jpeg')

if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ

    # ngrokのURLを取得
    url_info = requests.get('http://localhost:4040/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")

    app.run(port=5000)

In [ ]:
# ライブラリのインストール
!pip install flask flask-ngrok pyngrok flask-cors

from flask import Flask, request, send_file, jsonify, redirect
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask_cors import CORS
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time

# ngrokの認証トークンを設定
ngrok.set_auth_token("2xM5m67sPshbacPkiTIoQw5nnWb_7i5WdwHgUxf3TtKioJiG")

app = Flask(__name__)
CORS(app, resources={r"/process_images": {"origins": ["http://127.0.0.1:5501"]}})

# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    print(f"Downloading image from: {url}, Status code: {response.status_code}")
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")

@app.route('/process_images', methods=['POST'])
def process_images():
    try:
        data = request.json
        print(f"Received data: {data}")

        if not data:
            return jsonify({"error": "No data provided."}), 400

        image_urls = data.get("longshirt")
        if not image_urls or len(image_urls) != 4:
            return jsonify({"error": "Invalid input, expected 4 image URLs."}), 400

        image_points = [
        np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
        np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
        np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
        np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
    ]

        # 貼り付け先の画像のURL
        target_image_url = "https://ta03-jp.github.io/practice/longTshirts_frame_only.png"
        target_image = download_image(target_image_url)

        # target_points_list = [
        #     np.array([[210, 0], [205, 197], [95, 272], [9, 879], [422, 966], [816, 880], [731, 272], [622, 198], [618, 0]]),
        #     np.array([[1031, 17], [1023, 213], [911, 282], [839, 878], [1238, 953], [1644, 879], [1573, 281], [1462, 215], [1456, 19]]),
        #     np.array([[106, 1131], [0, 1203], [100, 1882], [487, 1885], [584, 1203], [342, 1082]]),
        #     np.array([[888, 1094], [699, 1203], [795, 1885], [1182, 1883], [1284, 1204], [1041, 1086]])
        # ]
        target_points_list = [
        np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
        np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
        np.array([[106, 1131],[0, 1203],[100, 1882],[487, 1885],[584, 1203],[342, 1082]]),
        np.array([[888, 1094],[699, 1203],[795, 1885],[1182, 1883],[1284, 1204],[1041, 1086]]),
    ]


        # pointsを定義する
        #points = np.array([[130, 117], [128, 171], [95, 193], [69, 381], [193, 408], [318, 382], [294, 195], [260, 170], [259, 117]])

        # 画像を処理する
        for url, target_points in zip(image_urls,image_points, target_points_list):
            print(f"Downloading image from: {url}")
            image = download_image(url)
            print(f"Successfully downloaded image: {url}")


            print(f"Source points for {url}: {points}, Shape: {points.shape}")
            print(f"Target points: {target_points}, Shape: {target_points.shape}")

            if len(points) != len(target_points):
                print(f"Mismatch in points count for {url}: {len(points)} vs {len(target_points)}")
                return jsonify({"error": f"Points count mismatch for {url}"}), 400

            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [points], 255)
            print(f"Mask shape: {mask.shape}, unique values in mask: {np.unique(mask)}")

            #M, _ = cv2.findHomography(points, target_points)
            M, status = cv2.findHomography(points, target_points)
            print(f"Homography Matrix for {url}:\n{M}")

            transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))
            print(f"Transformed image shape: {transformed_image.shape}")

            mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
            target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

        # 変更された画像を一時ファイルに保存
        modified_target_image_path = 'modified_image.jpg'
        cv2.imwrite(modified_target_image_path, target_image)

        # 生成された画像を返す
        return send_file(modified_target_image_path, mimetype='image/jpeg')

    except Exception as e:
        print(f"Error occurred: {e}")
        return jsonify({"error": str(e)}), 500

# テスト用app
@app.route('/', methods=['GET'])
def home():
    return "Flask is running! APIテストOK"

# JSテスト用app
@app.route('/img', methods=['GET'])
def img():
    image_url = "https://github.com/ta03-jp/muchu3/blob/main/longTshirts_frame_only.png"
    return redirect(image_url)

if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ

    # ngrokのURLを取得
    url_info = requests.get('http://localhost:4040/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")

    app.run(port=5000)


ngrok URL: https://962e-35-185-116-27.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 15:59:31] "OPTIONS /process_images HTTP/1.1" 200 -


Received data: {'longshirt': ['https://raw.githubusercontent.com/ta03-jp/muchu3/main/longshirt_front.jpg', 'https://raw.githubusercontent.com/ta03-jp/muchu3/main/longshirt_back.png', 'https://raw.githubusercontent.com/ta03-jp/muchu3/main/longshirt_sleeve.jpg', 'https://raw.githubusercontent.com/ta03-jp/muchu3/main/longshirt_sleeve.jpg']}


INFO:werkzeug:127.0.0.1 - - [28/Sep/2024 15:59:31] "POST /process_images HTTP/1.1" 500 -


Successfully downloaded image: https://raw.githubusercontent.com/ta03-jp/muchu3/main/longshirt_front.jpg
Error occurred: name 'i' is not defined


#静的ドメイン

In [ ]:
!pip install Flask pyngrok
from flask import Flask, request, send_file, jsonify
import cv2
import numpy as np
import requests
from io import BytesIO
from PIL import Image
import subprocess
import time

app = Flask(__name__)

# ngrokのトークンを設定
ngrok_token = "2xM5m67sPshbacPkiTIoQw5nnWb_7i5WdwHgUxf3TtKioJiG"  # ここにあなたのngrokトークンを入れてください
subprocess.run(['ngrok', 'authtoken', ngrok_token])

# GitHubから画像をダウンロードする関数
def download_image(url):
    url = url.replace("github.com", "raw.githubusercontent.com").replace("blob/", "")
    response = requests.get(url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise FileNotFoundError(f"画像が見つかりません: {url}")
# APIエンドポイント
@app.route('/')
def index():
    return "Welcome to the image processing API! Use the /process_images endpoint."
def process_images():
    data = request.json
    image_urls = data.get("longshirt")

    if not image_urls or len(image_urls) != 4:
        return jsonify({"error": "Invalid input, expected 4 image URLs."}), 400

    image_points = [
    np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
    np.array([[130, 117],[128, 171],[95, 193],[69, 381],[193, 408],[318, 382],[294, 195],[260, 170],[259, 117]]),
    np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]]),
    np.array([[135, 150],[103, 174],[133, 382],[256, 384],[286, 173],[212, 135]])
    ]

    # 貼り付け先の画像のURL
    target_image_url = "https://github.com/ta03-jp/practice/blob/main/longTshirts_frame_only.png"
    target_image = download_image(target_image_url)

    target_points_list = [
        np.array([[210, 0],[205, 197],[95, 272],[9, 879],[422, 966],[816, 880],[731, 272],[622, 198],[618, 0]]),
        np.array([[1031, 17],[1023, 213],[911, 282],[839, 878],[1238, 953],[1644, 879],[1573, 281],[1462, 215],[1456, 19]]),
        np.array([[106, 1131],[0, 1203],[100, 1882],[487, 1885],[584, 1203],[342, 1082]]),
        np.array([[888, 1094],[699, 1203],[795, 1885],[1182, 1883],[1284, 1204],[1041, 1086]]),
    ]

    # 画像を処理する
    for url, points, target_points in zip(image_urls, image_points, target_points_list):
        image = download_image(url)

        # マスクを作成
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [points], 255)

        M, _ = cv2.findHomography(points, target_points)
        transformed_image = cv2.warpPerspective(image, M, (target_image.shape[1], target_image.shape[0]))

        mask_transformed = cv2.warpPerspective(mask, M, (target_image.shape[1], target_image.shape[0]))
        target_image[mask_transformed > 0] = transformed_image[mask_transformed > 0]

    # 変更された画像を一時ファイルに保存
    modified_target_image_path = 'modified_image.jpg'
    cv2.imwrite(modified_target_image_path, target_image)

    # 生成された画像を返す
    return send_file(modified_target_image_path, mimetype='image/jpeg')

if __name__ == '__main__':
    # ngrokを起動する
    ngrok_process = subprocess.Popen(['ngrok', 'http','--domain=huge-nearly-puma.ngrok-free.app', '5000'])
    time.sleep(2)  # ngrokが立ち上がるまで待つ

    # ngrokのURLを取得

    url_info = requests.get('http://localhost:4040/api/tunnels').json()
    ngrok_url = url_info['tunnels'][0]['public_url']
    print(f"ngrok URL: {ngrok_url}")

    app.run(port=5000)